The NLP model that I will use to analyze emails to determine if they are internship-related or not

Need to go back through and include more examples of job related messages

In [ ]:
#Uncomment and run these cells if nltk is not downloaded
#Solution from https://stackoverflow.com/questions/38916452/nltk-download-ssl-certificate-verify-failed

#%pip install nltk
#import nltk
#import ssl

#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context
#
#nltk.download()

In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
import string

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


In [2]:
emails = pd.read_csv('my_emails.csv',header=0)

In [4]:
emails

,id,From,Subject,Body,Job,Status
0,1942dec531f3358c,Twilio Recruiting Team <support@untapped.io>,Reminder - Complete your Untapped profile for ...,"Hi Griffin, Thank you for applying to Twilio! ...",1,1
1,1942d25f123f13e1,DoorDash <no-reply@messages.doordash.com>,Get 60% off your next 3 orders plus 1 month free!,Join DashPass for 60% off your order. ͏; DoorD...,0,0
2,1942d209af881a8f,Etsy <email@email.etsy.com>,What’s our color of the year? 👀,Introducing Lime Cream! Etsy ( https://ablink....,0,0
3,1942cebdb640aaf0,Capital One <donotreply@cardmessage.capitalone...,Bonus opportunity: We're giving you an opportu...,Earn up to $500 per year! -- [ [Sign in to you...,0,0
4,1942ce4ef0ec44af,Costco Wholesale <Costco@digital.costco.com>,"New Year, New Storage Solutions- Shop Now!",NaN,0,0
...,...,...,...,...,...,...
178,193c66d43071c529,"""Capital One | Savor"" <capitalone@notification...",Your credit card payment is due,Make a payment. -- Capital One | Savor -- Savo...,0,0
179,193c61b602dfd38a,Costco Wholesale <Costco@digital.costco.com>,"Finish off your Shopping List with Apparel, Te...",NaN,0,0
180,193c60605c5abe45,"""Men's Wearhouse"" <menswearhouse@e.menswearhou...",The Holiday Sale ends soon. Don't wait.,Men's Wearhouse (http://l.e.menswearhouse.com/...,0,0
181,193c5c1a5cdaeecf,Etsy <email@email.etsy.com>,New message from Santa! 🎅🏻,Straight from the North Pole. Etsy ( https://a...,0,0


In [5]:
emails.dropna(inplace=True)

In [6]:
emails

,id,From,Subject,Body,Job,Status
0,1942dec531f3358c,Twilio Recruiting Team <support@untapped.io>,Reminder - Complete your Untapped profile for ...,"Hi Griffin, Thank you for applying to Twilio! ...",1,1
1,1942d25f123f13e1,DoorDash <no-reply@messages.doordash.com>,Get 60% off your next 3 orders plus 1 month free!,Join DashPass for 60% off your order. ͏; DoorD...,0,0
2,1942d209af881a8f,Etsy <email@email.etsy.com>,What’s our color of the year? 👀,Introducing Lime Cream! Etsy ( https://ablink....,0,0
3,1942cebdb640aaf0,Capital One <donotreply@cardmessage.capitalone...,Bonus opportunity: We're giving you an opportu...,Earn up to $500 per year! -- [ [Sign in to you...,0,0
5,1942913ad2e5ab1e,Amazon Web Services <no-reply-aws@amazon.com>,Amazon Web Services Billing Statement Availabl...,"Greetings from Amazon Web Services, This e-mai...",0,0
...,...,...,...,...,...,...
177,193c86108a206754,"""Amazon.com"" <store-news@amazon.com>","Up to 40% off presents, Our Top 100+ Gifts, an...",Up to 40% off gifts for all https://www.amazon...,0,0
178,193c66d43071c529,"""Capital One | Savor"" <capitalone@notification...",Your credit card payment is due,Make a payment. -- Capital One | Savor -- Savo...,0,0
180,193c60605c5abe45,"""Men's Wearhouse"" <menswearhouse@e.menswearhou...",The Holiday Sale ends soon. Don't wait.,Men's Wearhouse (http://l.e.menswearhouse.com/...,0,0
181,193c5c1a5cdaeecf,Etsy <email@email.etsy.com>,New message from Santa! 🎅🏻,Straight from the North Pole. Etsy ( https://a...,0,0


PreProcessing

In [7]:
from nltk.corpus import stopwords

In [8]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(emails['Body'])

In [15]:
emails_bow = bow_transformer.transform(emails['Body'])

In [16]:
print('Shape of Sparse Matrix: ', emails_bow.shape)
print('Amount of Non-Zero occurences: ', emails_bow.nnz)

Shape of Sparse Matrix:  (155, 8322)
Amount of Non-Zero occurences:  21164


Training the model

In [22]:
from sklearn.model_selection import train_test_split

mail_train, mail_test, job_train, job_test = \
train_test_split(emails['Body'], emails['Job'], test_size=.2)

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [25]:
pipeline.fit(mail_train, job_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x167e73ec0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [26]:
predictions = pipeline.predict(mail_test)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(predictions, job_test))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        31
           1       0.00      0.00      0.00         0

    accuracy                           0.87        31
   macro avg       0.50      0.44      0.47        31
weighted avg       1.00      0.87      0.93        31



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif